In [ ]:
import numpy as np
import keras
import datetime
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint, TensorBoard
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import LeakyReLU
import tensorflow as tf
from scipy import signal
import matplotlib.pyplot as plt
import IPython.display as ipd
from tqdm import tqdm
import h5py
from sklearn import preprocessing
from mpi4py import MPI
import json
from librosa.core import resample
import os
import audiofile as af
from functools import partial
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

In [ ]:
def normalize(x,save=False):
    scaled_x = (x - np.mean(x))/(np.abs(np.std(x))+1e-8)

    if save:
        return scaled_x, np.mean(x), np.std(x)
    return scaled_x
def normalize_from_outer(x,x_mean,x_std):
    scaled_x = (x - x_mean)/(x_std+1e-8)
    return scaled_x
def preprocess(sample,x_mean,x_std):
    log_sample = np.log(np.abs(sample)+1e-7)
    mod_input = normalize_from_outer(log_sample,x_mean,x_std)
    return mod_input
def denormalize(x,x_mean,x_std):
    scaled_x = x*(x_std + 1e-8) + x_mean
    return scaled_x

In [ ]:
# import musdb
# mus = musdb.DB('/nfs/home/pedro.lopes/data/dataset/musdb18hq')


In [ ]:
#tf.debugging.set_log_device_placement(True)
json_path = '../norm_data_full.json'
with open(json_path) as infile:
        norm_data = json.load(infile)
dataset_path = '/home/pedro.lopes/data/audio_data/train/augmented/'
X_mean = norm_data['X_min']
X_std = norm_data['X_max'] - norm_data['X_min']

def preprocess_audio(audio,rate,return_stft=False):
    samples = []
    num_samples = 20
    offset = 1500
    freq_bins = 512
    sample_len = 128
    stft_data = tf.signal.stft(audio, 
	frame_length=1024, 
	frame_step=512,
	fft_length=1024).numpy().T
    


    X_mean = norm_data['X_min']
    X_std = norm_data['X_max'] - norm_data['X_min']
    mod_input = np.zeros((int(sample_len*np.ceil(len(stft_data[0])/sample_len)),513),dtype = complex)
    mod_input[0:len(stft_data[0])]= np.array(stft_data.T)

    
    range_check = range(0,int(sample_len*np.ceil(len(stft_data[0])/sample_len)),sample_len)
    for i in range_check:
        samples.append(mod_input[i:i+sample_len,1:513].T.reshape(freq_bins,sample_len,1))
    mod_input_array = preprocess(np.array(samples),X_mean,X_std)
    if return_stft:
        return mod_input_array,stft_data,samples
    return mod_input_array

def normalize(x,save=False):
    scaled_x = (x - np.mean(x))/(np.abs(np.std(x))+1e-8)

    if save:
      return scaled_x, np.mean(x), np.std(x)
    return scaled_x
def normalize_from_outer(x,x_mean,x_std):
  scaled_x = (x - x_mean)/(x_std+1e-8)
  return scaled_x
def preprocess(sample,x_mean,x_std):
  log_sample = np.log(np.abs(sample)+1e-7)
  mod_input = normalize_from_outer(log_sample,x_mean,x_std)
  return mod_input

def preprocess_tf(sample,x_mean,x_std):
  log_sample = tf.math.log(tf.math.abs(sample)+1e-7)
  mod_input = normalize_from_outer(log_sample,x_mean,x_std)
  return mod_input

def denormalize(x,x_mean,x_std):
  scaled_x = x*(x_std + 1e-8) + x_mean
  return scaled_x


In [ ]:
FRAME_LENGHT = 4096
HOP_SIZE =1024
N_BINS = 2048
# def normalize(x,save=False):
#     scaled_x = (x - np.mean(x))/(np.abs(np.std(x))+1e-8)

#     if save:
#       return scaled_x, np.mean(x), np.std(x)
#     return scaled_x
# def normalize_from_outer(x,x_mean,x_std):
#   scaled_x = (x - x_mean)/(x_std+1e-8)
#   return scaled_x
# def preprocess(sample,x_mean,x_std):
#   log_sample = np.log(np.abs(sample)+1e-7)
#   mod_input = normalize_from_outer(log_sample,x_mean,x_std)
#   return mod_input

# def preprocess_tf(sample,x_mean,x_std):
#   log_sample = tf.math.log(tf.math.abs(sample)+1e-7)
#   mod_input = normalize_from_outer(log_sample,x_mean,x_std)
#   return mod_input

# def denormalize(x,x_mean,x_std):
#   scaled_x = x*(x_std + 1e-8) + x_mean
#   return scaled_x


# def pad_tf(stft_data):
#     x_before = tf.transpose(stft_data)
#     x_shape = tf.cast(tf.shape(x_before)[-1],tf.float64)
#     print(x_shape)
#     pad_len = tf.math.floor(tf.math.ceil(x_shape/128)*128 - x_shape)
#     pad = ([0,0],[0,pad_len])
#     x = tf.pad(x_before,pad,mode='constant', constant_values=0)
#     x_split = tf.split(x,4,axis = -1)
#     return x_split

# def preprocess_audio_tf(x,y):
#     with tf.device('GPU:1'):
#         stft_data = tf.signal.stft(x, 
#         frame_length=FRAME_LENGHT, 
#         frame_step=HOP_SIZE,
#         fft_length=FRAME_LENGHT)
#         x = pad_tf(stft_data)
#         x = preprocess_tf(x,X_mean,X_std)[:,0:N_BINS]
#         x = tf.expand_dims(x,axis=-1)
#         stft_data = tf.signal.stft(y, 
#         frame_length=FRAME_LENGHT, 
#         frame_step=HOP_SIZE,
#         fft_length=FRAME_LENGHT)
#         y = pad_tf(stft_data)
#         y = preprocess_tf(y,X_mean,X_std)[:,0:N_BINS]
#         y = tf.expand_dims(y,axis=-1)
#     return x,y
# def pad_tf_test(stft_data,test=False):
#     x_before = tf.keras.backend.permute_dimensions(stft_data,(0,2,1))
#     x_shape = tf.cast(tf.shape(x_before)[-1],tf.float64)
#     pad_len = tf.math.floor(tf.math.ceil(x_shape/128)*128 - x_shape)
#     pad = ([0,0],[0,0],[0,pad_len])
#     x_padded = tf.pad(x_before,pad,mode='constant', constant_values=0)
#     if test:
#         x_split = tf.split(x_padded,int(x_padded.shape[-1]/128),axis = -1)
#     else:
#         x_split = tf.split(x_padded,4,axis = -1)
#     x = tf.concat(x_split,axis=0)
#     return x,x_padded


# def preprocess_audio_tf_test(l_input,test=False):
#     stft_data = tf.signal.stft(l_input, 
#       frame_length=FRAME_LENGHT, 
#       frame_step=HOP_SIZE,
#       fft_length=FRAME_LENGHT)
#     x,x_padded = pad_tf_test(stft_data,test)
#     x_padded = tf.squeeze(x_padded)
#     x = preprocess_tf(x,X_mean,X_std)[:,0:N_BINS]
#     x = tf.expand_dims(x,axis=-1)
#     if test==True:
#         return x,x_padded
#     return x

In [ ]:
# def split_stack_audio(audio):
#     audio_pad = np.pad(audio,(0,441000 - audio.shape[-1]%441000), 'constant', constant_values=0)
#     audio_input = np.stack(np.split(audio_pad,441000)).T
#     return audio_input


# def preprocess_audio(audio,rate):
#     samples = []
#     num_samples = 20
#     offset = 1500
#     freq_bins = 512
#     sample_len = 128
    
#     audio_split = split_stack_audio(audio)
#     audio_stft_list = []
#     for i in range(len(audio_split)):
#         stft_data = tf.signal.stft(audio_split[i], 
#         frame_length=1024, 
#         frame_step=512,
#         fft_length=1024).numpy().T
#         stft_list.append(audio)
#     stft_array = np.stack(stft_list)
#     json_path = '../norm_data_full.json'
#     with open(json_path) as infile:
#         norm_data = json.load(infile)
#     X_mean = norm_data['X_min']
#     X_std = norm_data['X_max'] - norm_data['X_min']
#     mod_input = np.zeros((int(sample_len*np.ceil(len(stft_data[0])/sample_len)),513),dtype = complex)
#     mod_input[0:len(stft_data[0])]= np.array(stft_data.T)
    
    
#     range_check = range(0,int(sample_len*np.ceil(len(stft_data[0])/sample_len)),sample_len)
#     for i in range_check:
#         samples.append(mod_input[i:i+sample_len,1:513].T.reshape(freq_bins,sample_len,1))
#     mod_input_array = preprocess(np.array(samples),X_mean,X_std)
#     return mod_input_array,stft_data,samples

def pad_tf(stft_data,test=False):
    x_before = tf.keras.backend.permute_dimensions(stft_data,(0,2,1))
    x_shape = tf.cast(tf.shape(x_before)[-1],tf.float64)
    pad_len = tf.math.floor(tf.math.ceil(x_shape/128)*128 - x_shape)
    pad = ([0,0],[0,0],[0,pad_len])
    x_padded = tf.pad(x_before,pad,mode='constant', constant_values=0)
    if test:
        x_split = tf.split(x_padded,int(x_padded.shape[-1]/128),axis = -1)
    else:
        x_split = tf.split(x_padded,7,axis = -1)
    x = tf.concat(x_split,axis=0)
    return x,x_padded

def preprocess_audio_tf(l_input,test=False):
    stft_data = tf.signal.stft(l_input, 
    frame_length=FRAME_LENGHT, 
    frame_step=HOP_SIZE,
    fft_length=FRAME_LENGHT)
    x,x_padded = pad_tf(stft_data,test)
    x_padded = tf.squeeze(x_padded)
    x = preprocess_tf(x,X_mean,X_std)[:,0:N_BINS]
    x = tf.expand_dims(x,axis=-1)
    if test==True:
        return x,x_padded
    return x

In [ ]:
def get_umx(input_spec,freq_bins,sample_len):
    DROPOUT = 0.4
    hidden_size = 512
    print(input_spec.shape)
    l1 = K.permute_dimensions(input_spec,(0,2,1,3))
    l1 = K.reshape(l1,(-1,freq_bins))
    #l1 = input_spec
    l2 = Dense(hidden_size)(l1)
    l2 = BatchNormalization()(l2)
    l2 = K.reshape(l2,(128,-1,hidden_size))


    l3 = Activation('tanh')(l2)
    l4 = K.reshape(l3,(sample_len,-1,hidden_size))
    #l4 = K.expand_dims(l4,axis=-1)
    x1 = Bidirectional(LSTM(int(hidden_size/2), return_sequences=True,dropout=DROPOUT))(l4)

    #x1 = Dropout(0.4)(x1)

    x2 = Bidirectional(LSTM(int(hidden_size/2), return_sequences=True,dropout=DROPOUT))(x1)
    #x2 = Dropout(0.4)(x2)
    
    x3 = Bidirectional(LSTM(int(hidden_size/2), return_sequences=True,dropout=DROPOUT))(x2)
    #x3 = Dropout(0.4)(x3)
    
    
    #x3 = tf.keras.layers.TimeDistributed(Dense(freq_bins))(x3)
    
    
    c1 = concatenate([l4,x3],axis=2)
    c1 = tf.reshape(c1,(-1,c1.shape[-1]))
    c2 = Dense(hidden_size)(c1)
    c2 = BatchNormalization()(c2)
    c3 = Activation('relu')(c2)
    c4 = Dense(freq_bins)(c3)
    c4 = BatchNormalization()(c4)
    c5 = Activation('relu')(c4)
    #c6 = K.permute_dimensions(c5,(1,0))
    c6 = K.reshape(c5,(-1,sample_len,freq_bins))
    c6 = K.permute_dimensions(c6,(0,2,1))
    c6 = K.reshape(c6,(-1,freq_bins,sample_len,1))
    return c6

In [ ]:
def get_unet_spleeter(input_tensor, kernel_size=(5, 5), strides=(2, 2)):
    DROPOUT = 0
    conv_activation_layer = LeakyReLU(0.2)
    deconv_activation_layer = ReLU()
    conv_n_filters = [16, 32, 64, 128, 256, 512]
    kernel_initializer = 'he_normal'
    conv2d_factory = partial(
        Conv2D, strides=strides, padding="same", kernel_initializer=kernel_initializer
    )
    # First layer.
    conv1 = conv2d_factory(conv_n_filters[0], kernel_size)(input_tensor)
    batch1 = BatchNormalization(axis=-1)(conv1)
    rel1 = conv_activation_layer(batch1)
    # Second layer.
    conv2 = conv2d_factory(conv_n_filters[1], kernel_size)(rel1)
    batch2 = BatchNormalization(axis=-1)(conv2)
    rel2 = conv_activation_layer(batch2)
    # Third layer.
    conv3 = conv2d_factory(conv_n_filters[2], kernel_size)(rel2)
    batch3 = BatchNormalization(axis=-1)(conv3)
    rel3 = conv_activation_layer(batch3)
    # Fourth layer.
    conv4 = conv2d_factory(conv_n_filters[3], kernel_size)(rel3)
    batch4 = BatchNormalization(axis=-1)(conv4)
    rel4 = conv_activation_layer(batch4)
    # Fifth layer.
    conv5 = conv2d_factory(conv_n_filters[4], kernel_size)(rel4)
    batch5 = BatchNormalization(axis=-1)(conv5)
    rel5 = conv_activation_layer(batch5)
    # Sixth layer
    conv6 = conv2d_factory(conv_n_filters[5], kernel_size)(rel5)
    batch6 = BatchNormalization(axis=-1)(conv6)
    _ = conv_activation_layer(batch6)
    #
    #
    conv2d_transpose_factory = partial(
        Conv2DTranspose,
        strides=strides,
        padding="same",
        kernel_initializer=kernel_initializer,
    )
    #
    up1 = conv2d_transpose_factory(conv_n_filters[4], kernel_size)((conv6))
    up1 = deconv_activation_layer(up1)
    batch7 = BatchNormalization(axis=-1)(up1)
    drop1 = Dropout(DROPOUT)(batch7)
    merge1 = Concatenate(axis=-1)([conv5, drop1])
    #
    up2 = conv2d_transpose_factory(conv_n_filters[3], kernel_size)((merge1))
    up2 = deconv_activation_layer(up2)
    batch8 = BatchNormalization(axis=-1)(up2)
    drop2 = Dropout(DROPOUT)(batch8)
    merge2 = Concatenate(axis=-1)([conv4, drop2])
    #
    up3 = conv2d_transpose_factory(conv_n_filters[2], kernel_size)((merge2))
    up3 = deconv_activation_layer(up3)
    batch9 = BatchNormalization(axis=-1)(up3)
    drop3 = Dropout(DROPOUT)(batch9)
    merge3 = Concatenate(axis=-1)([conv3, drop3])
    #
    up4 = conv2d_transpose_factory(conv_n_filters[1], kernel_size)((merge3))
    up4 = deconv_activation_layer(up4)
    batch10 = BatchNormalization(axis=-1)(up4)
    merge4 = Concatenate(axis=-1)([conv2, batch10])
    #
    up5 = conv2d_transpose_factory(conv_n_filters[0], kernel_size)((merge4))
    up5 = deconv_activation_layer(up5)
    batch11 = BatchNormalization(axis=-1)(up5)
    merge5 = Concatenate(axis=-1)([conv1, batch11])
    #
    up6 = conv2d_transpose_factory(1, kernel_size)((merge5))
    up6 = deconv_activation_layer(up6)
    batch12 = BatchNormalization(axis=-1)(up6)
    # Last layer to ensure initial shape reconstruction.

    up7 = Conv2D(
        1,
        (4, 4),
        dilation_rate=(2, 2),
        activation="sigmoid",
        padding="same",
        kernel_initializer=kernel_initializer,
    )((batch12))
    return up7

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size = (5,5),stride = 1, batchnorm = False):
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = kernel_size,\
              kernel_initializer = 'he_normal', padding = 'same', strides=(stride, stride))(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation(tf.nn.leaky_relu)(x)
    
#     # second layer
#     x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
#               kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
#     if batchnorm:
#         x = BatchNormalization()(x)
#     x = Activation(tf.nn.leaky_relu)(x)
    
    return x

def conv2d_transpose_block(input_tensor,n_filters,kernel_size = (5,5),strides = (2,2), batchnorm = False):
    x = Conv2DTranspose(n_filters, kernel_size = kernel_size, strides = strides, padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation(tf.nn.relu)(x)
    
    return x


def get_unet_mask(input_img, n_filters=16, dropout=0.5, kernel_size=(5, 5), batchnorm=True):

    # Contracting Path 1
    c1 = conv2d_block(input_img, n_filters * 1,
                      kernel_size=kernel_size, batchnorm=batchnorm, stride=2)
    #c1 = MaxPooling2D((2, 2))(c1)
    #c1 = Dropout(dropout)(c1)

    c2 = conv2d_block(c1, n_filters * 2, kernel_size=kernel_size,
                      batchnorm=batchnorm, stride=2)
    #c2 = MaxPooling2D((2, 2))(c2)
    #c2 = Dropout(dropout)(c2)

    c3 = conv2d_block(c2, n_filters * 4, kernel_size=kernel_size,
                      batchnorm=batchnorm, stride=2)
    #c3 = MaxPooling2D((2, 2))(c3)
    #c3 = Dropout(dropout)(c3)

    c4 = conv2d_block(c3, n_filters * 8, kernel_size=kernel_size,
                      batchnorm=batchnorm, stride=2)
    #c4 = MaxPooling2D((2, 2))(c4)
    #c4 = Dropout(dropout)(c4)

    c5 = conv2d_block(c4, n_filters=n_filters * 16,
                      kernel_size=kernel_size, batchnorm=batchnorm, stride=2)
    #c5= MaxPooling2D((2, 2))(c5)
    #c5 = Dropout(dropout)(c5)

    c6 = conv2d_block(c5, n_filters=n_filters * 32,
                      kernel_size=kernel_size, batchnorm=batchnorm, stride=2)

    # Expansive Path 1
    u3 = conv2d_transpose_block(c6, n_filters=n_filters * 16,
                                kernel_size=kernel_size, strides=(2, 2), batchnorm=batchnorm)
    u5 = concatenate([u3, c5])
    u5 = Dropout(dropout)(u5)

    u61 = conv2d_transpose_block(
        u5, n_filters=n_filters * 8, kernel_size=kernel_size, strides=(2, 2), batchnorm=batchnorm)
    u6 = concatenate([u61, c4])
    u6 = Dropout(dropout)(u6)
    #c6 = conv2d_block(u6, n_filters * 8, kernel_size = 5, batchnorm = batchnorm)

    u71 = conv2d_transpose_block(
        u6, n_filters=n_filters * 4, kernel_size=kernel_size, strides=(2, 2), batchnorm=batchnorm)
    u7 = concatenate([u71, c3])
    u7 = Dropout(dropout)(u7)
    #c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)

    u81 = conv2d_transpose_block(
        u7, n_filters=n_filters * 2, kernel_size=kernel_size, strides=(2, 2), batchnorm=batchnorm)
    u8 = concatenate([u81, c2])
    #u8 = Dropout(dropout)(u8)
    #c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)

    u91 = conv2d_transpose_block(
        u8, n_filters=n_filters * 1, kernel_size=kernel_size, strides=(2, 2), batchnorm=batchnorm)
    u9 = concatenate([u91, c1])
    #u9 = Dropout(dropout)(u9)
    #c9 = conv2d_block(u9, n_filters * 1, kernel_size = 5, batchnorm = batchnorm, stride=2)
    c9 = Conv2DTranspose(1, kernel_size=kernel_size, strides=(
        2, 2), padding='same', activation='sigmoid')(u9)
    #mask_layer = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    #l_out = Multiply()([l_input,mask_layer])
    #model = Model(inputs=[input_img], outputs=[l_out,l_out_2])
    return c9
    #return mask_layer, l_out
with tf.device('GPU:1'):
    init = keras.initializers.glorot_normal(seed=None)
    reg = 5e-5
    regularizer = tf.keras.regularizers.l2(reg)
    freq_bins = 2048
    sample_len = 128
    l_input = Input(shape=(freq_bins, 128, 1))
#     l_out_1 = get_unet_spleeter(l_input, kernel_size=(7, 3))
#     l_out_2 = get_unet_spleeter(l_input, kernel_size=(3, 7))
#     concat_layer = concatenate([l_out_1, l_out_2])
#     mask_layer = Conv2D(1, (1, 1))(concat_layer)
#     final_layer = Multiply()([l_input, mask_layer])
#     mask_layer= get_unet_spleeter(l_input, kernel_size=(5, 5))
#     final_layer = Multiply()([l_input, mask_layer])
    mask_layer= get_umx(l_input, 2048,128)
    final_layer = Multiply()([l_input, mask_layer])
#     l_input = Input(shape=(None,))
#     final_layer,mask_layer = get_unet_mask(l_input,kernel_size = (5,5))

#     l_out_2= get_unet_mask(l_input,kernel_size = (2,5))

    #l_out_1_2= get_unet_mask(l_input,kernel_size = (5,2))
    #l_out_2_2= get_unet_mask(l_input,kernel_size = (2,5))

    #concat_layer_2 = concatenate([l_out_1_2,l_out_2_2])
    #mask_layer_2 = Conv2D(1, (1, 1), activation='sigmoid')(concat_layer_2)
    #final_layer_2 = Multiply()([l_input,mask_layer_2])

    model = Model(inputs=[l_input], outputs=[final_layer])
    #model.load_weights('../checkpoints/umx_globo.hdf5')

In [ ]:
model.summary()

In [ ]:
FRAME_LENGHT = 4096
HOP_SIZE =1024
N_BINS = 2048
def separate_from_audio(audio,rate,mask_model,wiener_filter=True, return_spectrogram = False):
    split_stft, full_stft = preprocess_audio_tf(np.expand_dims(audio,axis=0),test=True)
    mask = mask_model.predict(split_stft)
    #objective = preprocess(np.array(np.hstack(objective_vocal_samples)),X_mean,X_std)
    mask_in_shape = np.concatenate(mask,axis=1)[:,:,0]
    input_in_shape = full_stft
    json_path = '../norm_data_full.json'
    with open(json_path) as infile:
        norm_data = json.load(infile)
    X_mean = norm_data['X_min']
    X_std = norm_data['X_max'] - norm_data['X_min']
    test_sample = np.zeros((N_BINS + 1,input_in_shape.shape[1]), dtype=complex)
    test_sample[0:N_BINS + 1] = full_stft[0:N_BINS + 1]
    mask_final = np.zeros((N_BINS + 1,test_sample.shape[1]))
    final_mag = np.zeros((N_BINS + 1,test_sample.shape[1]))

    mask_final[0:N_BINS] = np.concatenate(mask,axis=1)[:,:,0]
    pre_result= preprocess(test_sample,X_mean,X_std)


    final_mag = denormalize(mask_final,X_mean,X_std)
    result_stft = np.multiply(np.exp(final_mag),np.exp(1j*np.angle(test_sample)))

    audio_vocal_pred = tf.signal.inverse_stft(result_stft.T,frame_length=FRAME_LENGHT, 
    frame_step=HOP_SIZE,
    fft_length=FRAME_LENGHT,
    window_fn=tf.signal.inverse_stft_window_fn(N_BINS)).numpy()
    if wiener_filter:
        test_sample_T = test_sample.T[:,:,np.newaxis]
        result_stft_T = result_stft.T[:,:,np.newaxis,np.newaxis]


        v = norbert.contrib.residual_model(np.abs(result_stft_T),test_sample_T)
        result_wiener = norbert.wiener(v,test_sample_T,iterations=2)[:,:,:,0]
        result_stft = result_wiener.T.reshape(final_mag.shape[0],final_mag.shape[1])
        audio_vocal_pred = tf.signal.inverse_stft(result_stft.T,frame_length=FRAME_LENGHT, 
        frame_step=HOP_SIZE,
        fft_length=FRAME_LENGHT,
        window_fn=tf.signal.inverse_stft_window_fn(N_BINS)).numpy()

    
    if return_spectrogram:
        return result_stft,mask_in_shape,preprocess_tf(input_in_shape,X_mean,X_std)
    return audio_vocal_pred

In [ ]:
from glob import glob

number=41
filenames_mix = glob('/nfs/home/pedro.lopes/data/audio_data/train/val/clean/mix/*')
filenames_vocal = glob('/nfs/home/pedro.lopes/data/audio_data/train/val/clean/vocals/*')
audio,sr = af.read(filenames_mix[number])
#audio,sr = af.read('/nfs/home/pedro.lopes/data/audio_data/train/augmented/val/sample_1_mix.wav')
stft_data = tf.signal.stft(audio, 
    frame_length=FRAME_LENGHT, 
    frame_step=HOP_SIZE,
    fft_length=FRAME_LENGHT)

json_path = '../norm_data_full.json'
with open(json_path) as infile:
    norm_data = json.load(infile)
X_mean = norm_data['X_min']
X_std = norm_data['X_max'] - norm_data['X_min']
audio_vocal,sr = af.read(filenames_vocal[number])
#audio_vocal, sr = af.read('/nfs/home/pedro.lopes/data/audio_data/train/val/clean/sample_1_vocals.wav')
stft_data = tf.signal.stft(audio_vocal, 
    frame_length=FRAME_LENGHT, 
    frame_step=HOP_SIZE,
    fft_length=FRAME_LENGHT)
objective = np.squeeze(preprocess_tf(stft_data,X_mean,X_std)).T
#audio +=audio_vocal
import norbert
%matplotlib inline
plt.rcParams['figure.figsize'] = (24.0, 10.0)
fig, axes = plt.subplots(ncols=2,nrows=2)
#axes[1,0].pcolormesh(np.abs(test[0,:,:,0]))
sample_rate=44100
with tf.device('GPU:1'):
    mask_model = Model(l_input, mask_layer)

    result_stft,mask_in_shape,input_in_shape = separate_from_audio(audio,sample_rate,model,wiener_filter=True,return_spectrogram=True)
axes[1,0].pcolormesh(np.log(np.abs(result_stft)+1e-7))
axes[0,1].pcolormesh(mask_in_shape)
axes[1,1].pcolormesh(objective)
axes[0,0].pcolormesh(input_in_shape)

axes[1,0].set_title('Input mascarado')
axes[0,1].set_title('Máscara')
axes[1,1].set_title('Ground Truth')
axes[0,0].set_title('Input')
plt.show()

In [ ]:
ipd.Audio(audio,rate=sample_rate)

In [ ]:
#audio = data
audio_vocal_pred = separate_from_audio(audio,sample_rate,model,wiener_filter=True)
ipd.Audio(audio_vocal_pred,rate=sample_rate)

In [ ]:
audio_acc = audio - audio_vocal_pred[0:len(audio)]
ipd.Audio(audio_acc,rate=sample_rate)

In [ ]:
model.summary()

In [ ]:
import pandas as pd
table=pd.DataFrame(columns=["Name","Layer type","Output shape", "Number of parameters"])
for layer in model.layers:
    try:
        table = table.append({"Name":layer.name, "Layer type": layer.__class__.__name__,"Output shape":layer.output_shape, "Number of parameters": layer.count_params()}, ignore_index=True)
    except: 
        pass
table#.to_csv('original_unet.csv')

In [ ]:
test_signal = tf.ones_like(audio)
stft_data = tf.signal.stft(test_signal, 
    frame_length=1024, 
    frame_step=512,
    fft_length=1024)
output = tf.signal.inverse_stft(stft_data, 
    frame_length=1024, 
    frame_step=512,
    window_fn=tf.signal.inverse_stft_window_fn(512))
plt.plot(output[50000:60000])

In [ ]:
import mir_eval
alpha = 1
audio_acc_pred = audio - alpha*audio_vocal_pred[0:len(audio)]
reference_sources = np.asarray([audio_vocal,audio_acc_orig])
estimated_sources = np.asarray([audio_vocal_pred[0:len(audio)],audio_acc_pred])
_,sir_old,_,_ = mir_eval.separation.bss_eval_sources(reference_sources, estimated_sources)
num_iter = 100
step = 0.01
derivative = 0.3
tol=1e-4
for i in range(num_iter):
    alpha_new = alpha + derivative*step
    #alpha_new = alpha - step
    audio_acc_pred = audio - alpha_new*audio_vocal_pred[0:len(audio)]
    reference_sources = np.asarray([audio_vocal_orig,audio_acc_orig])
    estimated_sources = np.asarray([audio_vocal_pred[0:len(audio)],audio_acc_pred])
    _,sir_new,_,_ = mir_eval.separation.bss_eval_sources(reference_sources, estimated_sources)
    delta = (sir_new[1] - sir_old[1])
    derivative = delta/(alpha_new-alpha)
    print(derivative,alpha_new,sir_new[1])
    alpha = alpha_new
    if abs(delta)<= tol:
        print('achei!')
        break
    sir_old = sir_new
    
print('terminei')

In [ ]:
pattern = "/home/pedro.lopes/data/audio_data/train/augmented/train/*/*/*"
names = glob(pattern)
names_corrected =[]
for name in names:
    splitted = name.rsplit('_',maxsplit=1)[0]
    names_corrected.append(splitted)

paths = list(dict.fromkeys(names_corrected))
paths_tuples = [(path + '_mix.wav', path + '_vocals.wav') for path in paths]

number = 10
paths_tuples

In [ ]:
audio_acc_pred = audio - alpha_new*audio_vocal_pred[0:len(audio)]
ipd.Audio(audio_acc_pred,rate=44100)

In [ ]:
def get_NSDR(sdr,audio_mixed,audio_vocal):
    reference_sources = np.asarray([audio_vocal])
    estimated_sources = np.asarray([audio_mixed])

    sdr_base, _,_,_ = mir_eval.separation.bss_eval_sources(reference_sources, estimated_sources)
    return sdr - sdr_base

In [ ]:
import mir_eval
range_sdr = range(100,150,1)
sdr = np.zeros((50,2))
sir = np.zeros((50,2))
sar = np.zeros((50,2))
nsdr = np.zeros((50,2))
for i in tqdm(range_sdr):
    audio = mus[i].audio[:,0] + mus[i].audio[:,1]
    vocal_target = mus[i].targets['vocals'].audio[:,0] + mus[i].targets['vocals'].audio[:,1]
    acc_target = mus[i].targets['accompaniment'].audio[:,0] + mus[i].targets['accompaniment'].audio[:,1]
    reference_sources = np.asarray([vocal_target,acc_target])
    audio_vocal_pred = separate_from_audio(audio,rate,mask_model)[0:len(audio)]
    print(len(audio_vocal_pred))
    audio_acc_pred = audio - audio_vocal_pred[0:len(audio)]
    estimated_sources = np.asarray([audio_vocal_pred,audio_acc_pred])
    (sdr[i-100,:], sir[i-100,:], sar[i-100,:], perm) = mir_eval.separation.bss_eval_sources(reference_sources, estimated_sources)
    nsdr[i-100,0] = get_NSDR(sdr[i-100,0],audio,vocal_target)
    nsdr[i-100,1] = get_NSDR(sdr[i-100,1],audio,acc_target)
print('terminei! ')


In [ ]:
print('Results: ')
print('vocal mean SDR: ', np.mean(sdr[:,0]))
print('vocal mean NSDR: ', np.mean(nsdr[:,0]))
print('vocal mean SIR: ', np.mean(sir[:,0]))
print('vocal mean SAR: ', np.mean(sar[:,0]))
print('acc mean SDR: ', np.mean(sdr[:,1]))
print('acc mean NSDR: ', np.mean(nsdr[:,1]))
print('acc mean SIR: ', np.mean(sir[:,1]))
print('acc mean SAR: ', np.mean(sar[:,1]))

In [ ]:
def get_umx(input_spec,freq_bins,sample_len):
    DROPOUT = 0.4
    hidden_size = 512
    print(input_spec.shape)
    l1 = K.permute_dimensions(input_spec,(0,2,1,3))
    l1 = K.reshape(l1,(-1,freq_bins))
    #l1 = input_spec
    l2 = Dense(hidden_size)(l1)
    l2 = BatchNormalization()(l2)
    l2 = K.reshape(l2,(128,-1,hidden_size))


    l3 = Activation('tanh')(l2)
    l4 = K.reshape(l3,(sample_len,-1,hidden_size))
    #l4 = K.expand_dims(l4,axis=-1)
    x1 = Bidirectional(LSTM(int(hidden_size/2), return_sequences=True,dropout=DROPOUT))(l4)

    #x1 = Dropout(0.4)(x1)

    x2 = Bidirectional(LSTM(int(hidden_size/2), return_sequences=True,dropout=DROPOUT))(x1)
    #x2 = Dropout(0.4)(x2)
    
    x3 = Bidirectional(LSTM(int(hidden_size/2), return_sequences=True,dropout=DROPOUT))(x2)
    #x3 = Dropout(0.4)(x3)
    
    
    #x3 = tf.keras.layers.TimeDistributed(Dense(freq_bins))(x3)
    
    
    c1 = concatenate([l4,x3],axis=2)
    c1 = tf.reshape(c1,(-1,c1.shape[-1]))
    c2 = Dense(hidden_size)(c1)
    c2 = BatchNormalization()(c2)
    c3 = Activation('relu')(c2)
    c4 = Dense(freq_bins)(c3)
    c4 = BatchNormalization()(c4)
    c5 = Activation('relu')(c4)
    #c6 = K.permute_dimensions(c5,(1,0))
    c6 = K.reshape(c5,(-1,sample_len,freq_bins))
    c6 = K.permute_dimensions(c6,(0,2,1))
    c6 = K.reshape(c6,(-1,freq_bins,sample_len,1))
    return c6
    

In [ ]:
from tqdm import tqdm
for name in tqdm(names):
    basename = name.rsplit("_",1)[0]
    sample_no = name.rsplit("_",2)[1]
    src_voc = basename + '_vocals.wav'
    src_mix = basename + '_mix.wav'
    final_path_voc = sample_no + '/' + 'vocals.wav'
    final_path_mix = sample_no + '/' + 'mix.wav'
    !mkdir $sample_no
    !mv $src_voc $final_path_voc
    !mv $src_mix $final_path_mix